In [ ]:
!pip uninstall -y transformers
!pip install --upgrade --no-cache-dir transformers==4.36.2

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 176.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [ ]:
import os
print("File exists:", os.path.exists("/content/structured_dataset_with_ast1.json"))


File exists: True


In [ ]:
import json
from datasets import Dataset

# ✅ Define fixed file path
json_file = "/content/structured_dataset_with_ast1.json"

# ✅ Load JSON data
with open(json_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# ✅ Optional safety: filter entries that have both required keys
filtered_data = [entry for entry in raw_data if "combined_input" in entry and "comment" in entry]

# ✅ Extract input and target fields
inputs = [entry["combined_input"] for entry in filtered_data]
labels = [entry["comment"] for entry in filtered_data]

# ✅ Convert to Hugging Face Dataset format
dataset = Dataset.from_dict({"input": inputs, "target": labels})

print("✅ Dataset loaded with", len(dataset), "valid entries")


✅ Dataset loaded with 57965 valid entries


In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

model_name = "Salesforce/codet5-small"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `for

In [ ]:
def tokenize_function(example):
    input_enc = tokenizer(example["input"], truncation=True, padding="max_length", max_length=512)
    target_enc = tokenizer(example["target"], truncation=True, padding="max_length", max_length=64)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/57965 [00:00<?, ? examples/s]

In [ ]:
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split["train"]
eval_dataset = split["test"]


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/codet5-comment-gen",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="/content/logs",
    logging_strategy="epoch",
    report_to="none"
)
print("✅ TrainingArguments loaded successfully!")

✅ TrainingArguments loaded successfully!


In [ ]:
!pip uninstall -y peft
!pip install --upgrade --no-cache-dir peft==0.7.1

Found existing installation: peft 0.7.1
Uninstalling peft-0.7.1:
  Successfully uninstalled peft-0.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --no-cache-dir accelerate==0.25.0

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


In [ ]:
from transformers import Trainer
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.468300,1.105411
2,1.170400,1.025340
3,1.080200,0.997979


TrainOutput(global_step=19563, training_loss=1.2396495257309716, metrics={'train_runtime': 6159.5566, 'train_samples_per_second': 25.408, 'train_steps_per_second': 3.176, 'total_flos': 2.118153329757389e+16, 'train_loss': 1.2396495257309716, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/codet5-comment-gen-final")
tokenizer.save_pretrained("/content/codet5-comment-gen-final")

print("✅ Training complete!")

✅ Training complete!


In [ ]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.9979786276817322, 'eval_runtime': 73.9818, 'eval_samples_per_second': 78.357, 'eval_steps_per_second': 9.8, 'epoch': 3.0}
